In [1]:
import numpy as np
import pandas as pd
import jieba



In [4]:
# 加载停用词
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as file:
    # 注意 [:-1] 去掉换行符
    stopwords = [i[:-1] for i in file.readlines()]


In [7]:
# 加载 news
news = pd.read_csv('sqlResult.csv', encoding='gb18030')
print(news.shape)
print(news.head(5))

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [9]:
# 处理缺失值
# print(news[news.content.isna()].head(5))
news = news.dropna(subset=['content'])
print(news.shape)

(87054, 7)


In [10]:
# 分词
def split_text(text):
    text = text.replace(' ', '')
    text = text.replace('\n', '')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result
print(news.iloc[0].content)
print(split_text(news.iloc[0].content))

Building prefix dict from the default dictionary ...


此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。



Dumping model to file cache C:\Users\zero\AppData\Local\Temp\jieba.cache
Loading model cost 1.106 seconds.
Prefix dict has been built successfully.


 当然 MIUI9 确切 信息 等待 官方消息


In [11]:
import os
import pickle as pkl

if not os.path.exists('corpus.pkl'):
    corpus = list(map(split_text, [str(i) for i in news.content]))
    print(corpus[0])
    print(len(corpus))
    print(corpus[1])
    with open('corpus.pkl', 'wb') as file:
        pkl.dump(corpus, file)
else:
    with open('corpus.pkl', 'rb') as file:
        corpus = pkl.load(file)



 当然 MIUI9 确切 信息 等待 官方消息
87054
 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台


In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# 计算关键词词频
countvectorizer = CountVectorizer(encoding='gb18030', min_df=0.015)
tfidftransformer = TfidfTransformer()

countvector = countvectorizer.fit_transform(corpus)
print(countvector.shape)

(87054, 884)


In [16]:
# tf-idf 计算
tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf.shape)

(87054, 884)


In [18]:
# 标记是否是 新华社的新闻
label = list(map(lambda source: 1 if '新华' in str(source) else 0, news.source))

In [19]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import MultinomialNB
# 数据集切分
x_train, x_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size=0.3, random_state=42)
clf = MultinomialNB()

clf.fit(X=x_train, y=y_train)


MultinomialNB()

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_predict = clf.predict(x_test)
def show_test_result(y_true, y_pred):
    print('accuracy:',accuracy_score(y_true,y_pred))
    print('precison:',precision_score(y_true,y_pred))
    print('recall:',recall_score(y_true,y_pred))
    print('f1_score:',f1_score(y_true,y_pred))
    
show_test_result(y_test, y_predict)

accuracy: 0.889726997740935
precison: 0.9623849539815926
recall: 0.9141011022424933
f1_score: 0.9376218323586745


In [22]:
# 使用模型检测抄袭新闻
prediction = clf.predict(tfidf.toarray())
labels = np.array(label)
# compare_news_index中有两列：prediction为预测，labels为真实值
compare_news_index = pd.DataFrame({'prediction': prediction, 'labels': labels})


In [24]:
# 找到疑似抄袭的新闻
copy_news_index = compare_news_index[(compare_news_index['prediction']==1) & (compare_news_index['labels'] == 0)].index
# 实际为新华社的新闻
xinhua_news_index = compare_news_index[(compare_news_index['labels'] == 1)].index
print('可能为copy的新闻条数：', len(copy_news_index))

可能为copy的新闻条数： 2818


In [30]:
# 使用K-Means, 对全量文档进行聚类
if not os.path.exists('label.pkl'):
    # 使用k-means聚类
    from sklearn.preprocessing import Normalizer
    from sklearn.cluster import KMeans
    normalizer = Normalizer()
    scaled_array = normalizer.fit_transform(tfidf.toarray())
    
    kmeans = KMeans(n_clusters=25, random_state=42, n_jobs=-1)
    k_labels = kmeans.fit_predict(scaled_array)
    with open('label.pkl', 'wb') as file:
        pkl.dump(k_labels, file)
    print(k_labels.shape)
    print(k_labels[0])
else:
    with open('label.pkl', 'rb') as file:
        k_labels = pkl.load(file)


c:\users\zero\pycharmprojects\test\venv\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


(87054,)
16


In [33]:
# 把文章聚类结果保存
if not os.path.exists('id_class.pkl'):
    id_class = {index: class_ for index, class_ in enumerate(k_labels)}
    with open('id_class.pkl', 'wb') as file:
        pkl.dump(id_class, file)
else:
    with open('id_class.pkl', 'rb') as file:
        id_class = pkl.load(file)
        

In [34]:
if not os.path.exists("class_id.pkl"):
    from collections import defaultdict
    # 创建你class_id字段，key为classId,value为文档index
    class_id = defaultdict(set)
    for index, class_ in id_class.items():
        # 只统计新华社发布的class_id
        if index in xinhua_news_index.tolist():
            class_id[class_].add(index)
    with open('class_id.pkl', 'wb') as file:
        pkl.dump(class_id, file)
else:
    with open('class_id.pkl', 'rb') as file:
        class_id = pkl.load(file)
        
            

In [35]:
# 输出每个类别的文档个数
count = 0
for k in class_id:
    print(count, len(class_id[k]))
    count += 1


0 12324
1 10368
2 6933
3 3789
4 3507
5 1850
6 1653
7 1738
8 728
9 2283
10 949
11 1731
12 1603
13 1474
14 1564
15 9749
16 1843
17 874
18 2124
19 2008
20 1358
21 2844
22 2800
23 1254
24 1507


In [36]:
from sklearn.metrics.pairwise import cosine_similarity
# 查找相似文本 （使用聚类的结果进行filter）
def find_similar_text(cpindex, top=10):
    dist_dict = {i: cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(), key=lambda x: x[1][0], reverse=True)[:top]


In [39]:
import editdistance

cpindex = 3352
print('是否在新华社', cpindex in xinhua_news_index)
print('是否在copy_news', cpindex in copy_news_index)
print('3134是否在新华社', 3134 in xinhua_news_index)
print('3134是否在copy_news', 3134 in copy_news_index)

是否在新华社 False
是否在copy_news True
3134是否在新华社 True
3134是否在copy_news False


In [40]:
similar_list = find_similar_text(cpindex)
print(similar_list)
print('怀疑抄袭:\n', news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = similar_list[0][0]
print('相似原文:\n', news.iloc[similar2].content)
# 求任意两篇文章的编辑距离 
print('编辑距离:',editdistance.eval(corpus[cpindex], corpus[similar2]))

[(3134, array([[0.96849134]])), (63511, array([[0.94643198]])), (29441, array([[0.94283416]])), (3218, array([[0.87621892]])), (980, array([[0.87535155]])), (29615, array([[0.86936328]])), (29888, array([[0.86215862]])), (64046, array([[0.85278235]])), (29777, array([[0.84875422]])), (63974, array([[0.73415212]]))]
怀疑抄袭:
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比

In [41]:
def find_similar_sentence(candidate, raw):
    similist = []
    cl = candidate.strip().split('。')
    ra = raw.strip().split('。')
    for c in cl:
        for r in ra:
            similist.append([c, r, editdistance.eval(c, r)])
    # 最相似的5个句子
    sort = sorted(similist, key=lambda x:x[2])[:5]
    for c, r, ed in sort:
        if c!='' and r!= '':
            print('怀疑抄袭句:{0}\n相似原句:{1}\n 编辑距离:{2}\n'.format(c,r,ed))
find_similar_sentence(news.iloc[cpindex].content, news.iloc[similar2].content)

怀疑抄袭句:从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内
相似原句:从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内
 编辑距离:0

怀疑抄袭句:其中，一二线城市同比涨幅回落尤其明显
相似原句:其中，一二线城市同比涨幅回落尤其明显
 编辑距离:0

怀疑抄袭句:据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点
相似原句:据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点
 编辑距离:0

怀疑抄袭句:二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个
相似原句:二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个
 编辑距离:0

